In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 133872 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redir

In [ ]:
%tensorflow_version 1.x

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/bert-paper/")

In [ ]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

# Training

In [ ]:
# Move data to google cloud data storage
!gsutil -m rm -r gs://paper/paper-bert-uncased-v0/

!python3 run_classifier.py \
    --data_dir=../data/ \
    --bert_config_file=gs://bert-eng/bert-base-uncased/bert_config.json \
    --vocab_file=gs://bert-eng/bert-base-uncased/vocab.txt \
    --output_dir=gs://paper/paper-bert-uncased-v0/ \
    --init_checkpoint=gs://bert-eng/bert-base-uncased/bert_model.ckpt \
    --do_lower_case=True \
    --features=title,keywords,abstract \ #title
    --max_seq_length=512 \
    --do_train=True \
    --do_eval=False \
    --num_train_epochs=10 \
    --train_batch_size=128 \
    --save_checkpoints_steps=1000 \
    --keep_checkpoint_max=1000 \
    --use_tpu=True \
    --tpu_name=$TPU_ADDRESS

# Evaluation

In [ ]:
!python3 run_classifier.py \
    --data_dir=../data/ \
    --bert_config_file=gs://bert-eng/bert-base-uncased/bert_config.json \
    --vocab_file=gs://bert-eng/bert-base-uncased/vocab.txt \
    --output_dir=gs://paper/paper-keywords+abstract-bert-uncased-v0 \
    --do_lower_case=True \
    --max_seq_length=128 \
    --features=title,keywords,abstract \
    --do_train=False \
    --do_eval=True \
    --eval_batch_size=128 \
    --save_checkpoints_steps=1000 \
    --keep_checkpoint_max=20 \
    --use_tpu=True \
    --tpu_name=$TPU_ADDRESS

# Testing

In [ ]:
!python3 run_classifier.py \
    --data_dir=../data/ \
    --bert_config_file=gs://bert-eng/bert-base-uncased/bert_config.json \
    --vocab_file=gs://bert-eng/bert-base-uncased/vocab.txt \
    --output_dir=gs://paper/paper-bert-uncased-v0 \
    --do_lower_case=True \
    --max_seq_length=128 \
    --features=title,keywords,abstract \
    --do_predict=True \
    --predict_checkpoint_path=gs://paper/paper-bert-uncased-v0/model.ckpt \
    --predict_batch_size=128 \
    --use_tpu=True \
    --tpu_name=$TPU_ADDRESS

In [ ]:
lines = []
for line in tf.gfile.Open("gs://paper/paper-bert-uncased-v0/test_results.tsv"):
    lines.append(line.strip())    

import numpy as np
probabilities = np.empty((len(lines), len(lines[0].split())))

for ii, line in enumerate(lines):
    probabilities[ii,:] = list(map(float, line.split()))

from run_classifier import PaperProcessor
prosessor = PaperProcessor(["title", "keywords", "abstract"])
journal_2_index = {journal: i for i, journal in enumerate(PaperProcessor().get_labels())}

import json
y_valid = []
for line in prosessor._read_jsonl("../data/springer-valid.jsonl"):
    y_valid.append(journal_2_index[line['journal'].lower()])
y_valid = np.asarray(y_valid)

print(np.mean(np.repeat(y_valid, 1).reshape(-1,1) == np.argsort(probabilities, axis=-1)[:,:-2:-1])*1)
print(np.mean(np.repeat(y_valid, 3).reshape(-1,3) == np.argsort(probabilities, axis=-1)[:,:-4:-1])*3)
print(np.mean(np.repeat(y_valid, 5).reshape(-1,5) == np.argsort(probabilities, axis=-1)[:,:-6:-1])*5)
print(np.mean(np.repeat(y_valid, 10).reshape(-1,10) == np.argsort(probabilities, axis=-1)[:,:-11:-1])*10)